<a href="https://colab.research.google.com/github/TerrorismAnalyticsBureau/TAB-AI/blob/main/TABAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import os
import chardet
from sklearn.preprocessing import LabelEncoder

# Set pyTorch local env to use segmented GPU memory
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear GPU cache & Set the device to use GPU
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
# Skip rows = 1 because those are the column names
X = np.array([])


# Load the data
# Open the file and detect the encoding
#with open('./globalterrorismdb_0718dist.csv', 'rb') as f:
#    result = chardet.detect(f.read())

# Get the detected encoding
#print(f"Detected encoding: {result['encoding']}")

# Now, read the file using the detected encoding
data = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding="Windows-1252")

# Extract relevant columns (adjust indices or column names as needed)
data_columns = data.iloc[:, [1, 2, 3, 7, 9]]
data_columns = data_columns.fillna(0)

target_columns = data.iloc[:, [1, 2, 3, 7, 9]]
target_columns = data_columns.fillna(0)

data_tensor = torch.tensor(data_columns.values, dtype=torch.float32)
target_tensor = torch.tensor(target_columns.values, dtype=torch.float32)

# Example list of groups
groups = []
for group in data['gname']:
    if group not in groups:
        groups.append(group)

provstates = []
for provstate in data['provstate']:
    if provstate not in provstates:
        provstates.append(provstate)

cities = []
for city in data['city']:
    if city not in cities:
        cities.append(city)

# Initialize label encoder
label_encoder = LabelEncoder()

# Fit the encoder and transform the group names into integers
group_numerical = label_encoder.fit_transform(groups)

# use cols skips over column 6 as that is the symbol of the stock
X_tensor = data_tensor
Y_tensor = target_tensor

# Set tensors to use GPU
X_tensor = X_tensor.to(device)
Y_tensor = Y_tensor.to(device)

print(X_tensor)

class LSTMAttackPredictor(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMAttackPredictor, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # Use the last time step output
        return out


model = LSTMAttackPredictor(input_size=5, hidden_size=128, output_size=3)
model  = model.to(device)


<ipython-input-7-a7f31e96fd7d>:32: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./globalterrorismdb_0718dist.csv', encoding="Windows-1252")


tensor([[1.9700e+03, 7.0000e+00, 2.0000e+00, 5.8000e+01, 2.0000e+00],
        [1.9700e+03, 0.0000e+00, 0.0000e+00, 1.3000e+02, 1.0000e+00],
        [1.9700e+03, 1.0000e+00, 0.0000e+00, 1.6000e+02, 5.0000e+00],
        ...,
        [2.0170e+03, 1.2000e+01, 3.1000e+01, 1.6000e+02, 5.0000e+00],
        [2.0170e+03, 1.2000e+01, 3.1000e+01, 9.2000e+01, 6.0000e+00],
        [2.0170e+03, 1.2000e+01, 3.1000e+01, 1.6000e+02, 5.0000e+00]],
       device='cuda:0')


name (csv column number)
```
•    iyear (1), imonth (2), iday (3)
•    country code (7) and country_txt (8)
•    region code (9) and region_txt (10)
•    provstate (11) and city (12)
•    latitude (13) and longitude (14)
•    attacktype1 (28) and attacktype1_txt (29)
•    targtype1 (34) and targtype1_txt (35)
•    targsubtype1 (36) and targsubtype1_txt (37)
•    target1 (39) (the specific target by name, building or person)

•    natity1 (40) and natity1_txt (41) (maybe later)
•    gname (group name) (57)
•    weaptype1 (81) and weaptype1_txt (82) (maybe)
•    nkill (98) and nwound (101)

```

